# Testing Connection to Google Sheets

In [1]:
import numpy as np
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import jsmith_acquire
import jsmith_prepare

In [2]:
#Set up credentials
gc = gspread.service_account(filename='credentials.json')

In [3]:
#Open Google Sheet By Name
gsheet = gc.open("Pending Reports")

In [4]:
#Try Loading First Tab, 'DistrictCivil', as a dataframe
original_data = pd.DataFrame(gsheet.sheet1.get_all_records())

In [5]:
original_data.head()

,cause_number,notes,disposed,dispose_date,on_track,file_date,cause_of_action,docket_date,docket_type,plaintiff
0,03-06-19302-MCV,Waiver of service 6/30/2003,,,,,,,,
1,21-12-40648-MCV,WAIVER OF SERVICE 12/09/2021,,,,12/8/2021,DIVORCE - NO CHILDREN,,,FIDEL MARTINEZ JR.
2,07-01-22238-MCV,WAIVER OF SERVICE 01/10/2022,,,,1/9/2007,"MOTION ENFORCEMENT, TO M",5/14/2007,FINAL HEARING,
3,03-08-19404-MCV,Waive 8/11/2003,,,,,,,,
4,15-07-31756-MCV,"UIFSA - FOREIGN SUPPORT ORDER, NOTICE OF REG. ...",FALSE,,,7/27/2015,1,,,"DEHOYOS, MARIO ALBERT"


# Connection Successful!

Now try adding the newly created data from your functions to the other tabs.

In [4]:
#Create a list of PDF paths to iterate through
path_list = ['/Users/johnathonsmith/Downloads/CV.PEND.CASE_Maverick_2-9-2022.pdf',
             '/Users/johnathonsmith/Downloads/CV.PEND.CASE_Dimmit_2-9-2022.pdf',
             '/Users/johnathonsmith/Downloads/CV.PEND.CASE_Zavala_2-9-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Zavala_2-9-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Maverick_2-9-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Dimmit_2-9-2022.pdf']

In [7]:
path_list[2]

'/Users/johnathonsmith/Downloads/CV.PEND.CASE_MAVERICK_2-3-2022.pdf'

In [13]:
#Try adding just one first
df = jsmith_acquire.build_dataframe(path_list[2])
df = jsmith_prepare.prepare_dataframe(df)

Collected Data From 1110 Cases.


In [14]:
df.head()

,County,Cause Number,File Date,Cause of Action,Docket Date,Docket Type,ANS File,CR Number,Plaintiff Name,Plaintiff Attorney,Defendant Name,Defendant Attorney,Case Type,On Track,Status,File Has Image,Need File,Disposed Date,Finding,Finding Date
0,Maverick,00-02-16392-CV,02/04/2000,(TITLE IV-D OAG USE ONLY,,,,,"ATTORNEY GENERAL\nCHAVARRIA, AZUCENA",MELINDA S. LAUCK,"CHAVARRIA, JOSE M.",,Civil,False,Pending,,,,,
1,Maverick,1-01-33838-MCV,01/11/2017,MOTOR VEHICLE,,,,,"CANO, ANA",ROLANDO SALINAS,"FIDLER, WILLIAM",,Civil,False,Pending,,,,,
2,Maverick,1-03-34076-MCV,03/09/2017,DIVORCE - CHILDREN,,,,,"MORALES, SYLVIA DANIELA",FELIPE HERNANDEZ,"MORALES, ROBERTO",,Civil,False,Pending,,,,,
3,Maverick,01-06-17430-CV,11/09/2017,SHOW CAUSE MOTION,08/29/2001,PETITION TO ESTA,,,"ATTORNEY GENERAL\nMUNOZ, CLARISA",MELINDA S. LAUCK,"BARRIENTOS, JOSE ALEJANDRO",,Civil,False,Pending,,,,,
4,Maverick,1-07-34542-MCV,07/17/2017,DIVORCE - CHILDREN,,,,,"RODRIGUEZ AVALOS, ALEJANDRO",FRANCISCO MARTINEZ III,"AVALOS, MARIELLA",,Civil,False,Pending,,,,,


In [5]:
#Civil cases go to the 'Civil Cases' tab
civil_sheet = gsheet.worksheet('Civil Cases')

#Criminal cases go to the 'Criminal Cases' tab
crim_sheet = gsheet.worksheet('Criminal Cases')

In [16]:
civil_sheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk',
 'updatedRange': "'Civil Cases'!A1:T1111",
 'updatedRows': 1111,
 'updatedColumns': 20,
 'updatedCells': 22220}

In [6]:
#Create dataframes
civil_df = pd.DataFrame()
crim_df = pd.DataFrame()

In [8]:
#Use a loop to extract all the data and prepare the dataframes accordingly.
#Add them one by one to the google sheet in the proper tabs

for path in path_list:
    #Extract the PDF data
    df = jsmith_acquire.build_dataframe(path)
    
    #Prepare the df and add new columns
    df = jsmith_prepare.prepare_dataframe(path, df)
    
    if df['Case Type'][0] == 'Criminal':
        #Add to criminal cases tab
        crim_df = crim_df.append(df, ignore_index = True)
    elif df['Case Type'][0] == 'Civil' or df['Case Type'][0] == 'Tax':
        civil_df = civil_df.append(df, ignore_index = True)
    else:
        print('Something went wrong in the loop!')

Collected Data From 1086 Cases.
Collected Data From 626 Cases.
Collected Data From 550 Cases.
Collected Data From 56 Cases.
Collected Data From 212 Cases.
Collected Data From 76 Cases.


In [9]:
#Now add it to the Criminal Cases tab in the Google Sheet
crim_sheet.update([crim_df.columns.values.tolist()] + crim_df.values.tolist())

{'spreadsheetId': '1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk',
 'updatedRange': "'Criminal Cases'!A1:O345",
 'updatedRows': 345,
 'updatedColumns': 15,
 'updatedCells': 5175}

In [10]:
#Next add the data to the Civil Cases tab
civil_sheet.update([civil_df.columns.values.tolist()] + civil_df.values.tolist())

{'spreadsheetId': '1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk',
 'updatedRange': "'Civil Cases'!A1:S2263",
 'updatedRows': 2263,
 'updatedColumns': 19,
 'updatedCells': 42997}

# How To Update the Data?

Now that I know how to push the data up to a spreadsheet, what's the best way to update it? Things to look for:

- Updated docket dates
- Updated docket types, although, I don't think this would change without there being a change in the docket date

I think I'll have to use the pd.drop_duplicates function twice. The first time, I'll use it to drop duplicates of subset ['Cause Number', 'Docket Date'] and keep the first instance. This means that if a docket date has not changed, the original work will be kept. The second time, I'll use it to drop duplicates of subset ['Cause Number'] and keep the last instance. This means that if there were duplicate cause numbers, with different docket dates, the instance with the later docket date will be kept. The only reason a docket date should change is if it had been recently updated, so we'll keep that version.

In [19]:
#Load the data currently on the civil cases tab in the 'Pending Reports' spreadsheet
civil_df = pd.DataFrame(civil_sheet.get_all_records())

In [20]:
civil_df.head()

,County,Cause Number,File Date,Cause of Action,Docket Date,Docket Type,ANS File,CR Number,Plaintiff Name,Plaintiff Attorney,Defendant Name,Defendant Attorney,Case Type,On Track,Status,File Has Image,Disposed Date,Finding,Finding Date
0,Dimmit,00-03-01927-TX,03/21/2000,TAX-BEFORE 1.1.2008,,,,,JOHN DAVID BRADY\nELIZABETH BRADY,,,,Tax,FALSE,Pending,,,,
1,Dimmit,00-04-01934-TX,04/20/2000,TAX SUIT,,,,,STATE OF TEXAS COUNTY OF DI,DANIEL P. O'CONNOR,ALICIA RANGEL GOMEZ,,Tax,FALSE,Pending,,,,
2,Dimmit,00-04-01938-TX,04/20/2000,TAX SUIT,,,,,STATE OF TEXAS COUNTY OF DI,JAMES E. CABELLO,LUCINDA LEDESMA\nPATRICIA LEDESMA\nM E GARY TR...,,Tax,FALSE,Pending,,,,
3,Dimmit,00-04-09284-CV,04/17/2000,"SUIT ON CONTRACTS,NOTES,",,,,,"FIRST SELECT, INC.",AUBYN JR. SHETTLE,CANDIDO R. DEANDA,,Civil,FALSE,Pending,,,,
4,Dimmit,00-05-01948-TX,01/19/2021,TAX DELINQUENCY,,,,,CARRIZO SPRINGS CONSOLIDATE,JAMES E. CABELLO,"GUMECINDO GUTIERREZ, ET. AL.",,Tax,FALSE,Pending,,,,


In [12]:
#Now Create New Civil case dataframe and append that data to the data above
new_civil_cases = jsmith_acquire.build_dataframe(path_list[0])
new_civil_cases = jsmith_prepare.prepare_dataframe(new_civil_cases)

Collected Data From 644 Cases.


In [21]:
new_civil_cases.head()

,County,Cause Number,File Date,Cause of Action,Docket Date,Docket Type,ANS File,CR Number,Plaintiff Name,Plaintiff Attorney,Defendant Name,Defendant Attorney,Case Type,On Track,Status,File Has Image,Disposed Date,Finding,Finding Date
0,Dimmit,00-03-01927-TX,03/21/2000,TAX-BEFORE 1.1.2008,,,,,JOHN DAVID BRADY\nELIZABETH BRADY,,,,Tax,False,Pending,,,,
1,Dimmit,00-04-01934-TX,04/20/2000,TAX SUIT,,,,,STATE OF TEXAS COUNTY OF DI,DANIEL P. O'CONNOR,ALICIA RANGEL GOMEZ,,Tax,False,Pending,,,,
2,Dimmit,00-04-01938-TX,04/20/2000,TAX SUIT,,,,,STATE OF TEXAS COUNTY OF DI,JAMES E. CABELLO,LUCINDA LEDESMA\nPATRICIA LEDESMA\nM E GARY TR...,,Tax,False,Pending,,,,
3,Dimmit,00-04-09284-CV,04/17/2000,"SUIT ON CONTRACTS,NOTES,",,,,,"FIRST SELECT, INC.",AUBYN JR. SHETTLE,CANDIDO R. DEANDA,,Civil,False,Pending,,,,
4,Dimmit,00-05-01948-TX,01/19/2021,TAX DELINQUENCY,,,,,CARRIZO SPRINGS CONSOLIDATE,JAMES E. CABELLO,"GUMECINDO GUTIERREZ, ET. AL.",,Tax,False,Pending,,,,


In [22]:
#append the data 
civil_df = civil_df.append(new_civil_cases, ignore_index = True)

In [23]:
#What are the duplicates? Should be pretty much everything from the dimmit county
civil_df['Cause Number'].value_counts()

00-03-01927-TX     2
19-06-02948-DTX    2
19-10-02994-DTX    2
19-10-02996-DTX    2
19-10-02998-DTX    2
                  ..
20-11-14658-ZCV    1
20-11-2482-TX      1
20-11-2478-TX      1
20-11-2468-TX      1
7815-OTH           1
Name: Cause Number, Length: 2533, dtype: int64

# Drop Duplicates For ['Cause Number', 'Docket Date']

The first stage is to drop duplicates for both Cause Number AND Docket Date, keeping the first. For this test run, I changed the docket dates for two of the cases in Dimmit County on the Pending Reports spreadsheet to the year 1900. I'll be using the original data for this test, so changing the docket date was to simulate the new data containing updated docket dates for those two cases. After this stage, those two should be the only two duplicates left.

Those cases are:

* 04-11-02272-DTX
* 02-11-02188-DTX

In [24]:
civil_df = civil_df.drop_duplicates(subset = ['Cause Number', 'Docket Date'], keep = 'first', ignore_index = True)

In [25]:
#There should only be two Cause Numbers left that are duplicated
civil_df['Cause Number'].value_counts()

04-11-02272-DTX    2
02-11-02188-DTX    2
00-03-01927-TX     1
18-06-05482-TX     1
18-05-35652-MCV    1
                  ..
11-09-12564-ZCV    1
11-09-12582-ZCV    1
11-09-12586-ZCV    1
11-10-12588-ZCV    1
7815-OTH           1
Name: Cause Number, Length: 2533, dtype: int64

# Drop Duplicates For ['Cause Number']

The second stage is to drop duplicates for Cause Number alone, keeping the second. After this, there should be no duplicates left and the docket dates should be updated for those two cases.

In [26]:
civil_df = civil_df.drop_duplicates(subset = ['Cause Number'], keep = 'last', ignore_index = True)

In [27]:
#There should be no duplicates left
civil_df['Cause Number'].value_counts()

00-03-01927-TX       1
18-06-35834-MCV      1
18-05-35670-MCV      1
18-05-35703-MCVAJ    1
18-06-05456-TX       1
                    ..
11-09-12582-ZCV      1
11-09-12586-ZCV      1
11-10-12588-ZCV      1
11-10-12594-ZCV      1
04-11-02272-DTX      1
Name: Cause Number, Length: 2533, dtype: int64

In [29]:
#Check the docket dates for the two cases I changed earlier
civil_df[civil_df['Cause Number'] == '04-11-02272-DTX']

,County,Cause Number,File Date,Cause of Action,Docket Date,Docket Type,ANS File,CR Number,Plaintiff Name,Plaintiff Attorney,Defendant Name,Defendant Attorney,Case Type,On Track,Status,File Has Image,Disposed Date,Finding,Finding Date
2532,Dimmit,04-11-02272-DTX,11/04/2004,TAX-BEFORE 1.1.2008,03/02/2021,TRIAL ON THE MER,,,DIMMIT COUNTY,CARLOS ARCE,MICHAEL DAVID FUGGER,,Tax,False,Pending,,,,


In [30]:
civil_df[civil_df['Cause Number'] == '02-11-02188-DTX']

,County,Cause Number,File Date,Cause of Action,Docket Date,Docket Type,ANS File,CR Number,Plaintiff Name,Plaintiff Attorney,Defendant Name,Defendant Attorney,Case Type,On Track,Status,File Has Image,Disposed Date,Finding,Finding Date
2531,Dimmit,02-11-02188-DTX,11/12/2002,TAX SUIT,11/02/2021,TRIAL ON THE MER,,,CITY OF CARRIZO SPRINGS,JAMES E. CABELLO,TOMAS PENA\nALICIA PENA\nJUAN M. RAMON (LIENHO...,,Tax,False,Pending,,,,


# Cases Successfully Updated!

I believe that's all I had to do to update the cases. I shouldn't have to worry about losing any previous work as cases are updated because the previous work shouldn't really apply anymore with the new docket date. The next step is to write it as a function.

In [43]:
def update_spreadsheet(path_list):
    """
    This function takes in a list containing the file paths of the PDFs that need to be extracted. It'll loop
    and build civil and criminal case dataframes. Then, it will load the data currently stored in the 'Pending Reports'
    google sheet and turn it into a dataframe. Finally, it will append the dataframes appropriately, drop duplicates,
    and then upload the updated data to the 'Pending Reports' google sheet.
    """
    #Set up dataframes for new data
    new_civil_df = pd.DataFrame()
    
    new_crim_df = pd.DataFrame()
    
    
    for path in path_list:
        #Extract the PDF data
        df = jsmith_acquire.build_dataframe(path)

        #Prepare the df and add new columns
        df = jsmith_prepare.prepare_dataframe(df)

        if df['Case Type'][0] == 'Criminal':
            #Add to criminal cases tab
            new_crim_df = crim_df.append(df, ignore_index = True)
        elif df['Case Type'][0] == 'Civil' or df['Case Type'][0] == 'Tax':
            new_civil_df = civil_df.append(df, ignore_index = True)
        else:
            print('Something went wrong in the loop!')
        
    #Set up credentials to interact with Google Sheets
    gc = gspread.service_account(filename='credentials.json')
    
    #Open 'Pending Reports' Google Sheet By Name
    gsheet = gc.open("Pending Reports")
    
    #Civil cases go to the 'Civil Cases' tab
    civil_sheet = gsheet.worksheet('Civil Cases')

    #Criminal cases go to the 'Criminal Cases' tab
    crim_sheet = gsheet.worksheet('Criminal Cases')
    
    #Load the data currently on the civil cases tab in the 'Pending Reports' spreadsheet
    current_civil_df = pd.DataFrame(civil_sheet.get_all_records())
    
    #Load the data currently on the criminal cases tab in the 'Pending Reports' spreadsheet
    current_crim_df = pd.DataFrame(crim_sheet.get_all_records())
    
    #Append new_civil_df to current_civil_df
    current_civil_df = current_civil_df.append(new_civil_df, ignore_index = True)
    
    #Append new_crim_df to current_crim_df
    current_crim_df = current_crim_df.append(new_crim_df, ignore_index = True)
    
    #Stage 1 - Drop Duplicates for subset ['Cause Number', 'Docket Date'] while keeping first
    #For civil cases
    current_civil_df = current_civil_df.drop_duplicates(subset = ['Cause Number', 'Docket Date'], ignore_index = True, keep = 'first')
    
    #For criminal cases
    current_crim_df = current_crim_df.drop_duplicates(subset = ['Cause Number', 'Docket Date'], ignore_index = True, keep = 'first')
    
    #Stage 2 - Drop Duplicates for subset ['Cause Number'] while keeping last
    #For civil cases
    current_civil_df = current_civil_df.drop_duplicates(subset = ['Cause Number'], ignore_index = True, keep = 'last')
    
    #For criminal cases
    current_crim_df = current_crim_df.drop_duplicates(subset = ['Cause Number'], ignore_index = True, keep = 'last')
    
    #Now sort by county and then by cause number
    #For civil cases
    current_civil_df = current_civil_df.sort_values(by = ['County', 'Cause Number'], ignore_index = True)
    
    #For criminal cases
    current_crim_df = current_crim_df.sort_values(by = ['County', 'Cause Number'], ignore_index = True)
    
    #Now upload to appropriate tabs in 'Pending Reports' spreadsheet and leave a message
    #For civil cases
    civil_sheet.update([current_civil_df.columns.values.tolist()] + current_civil_df.values.tolist())
    print('Civil Cases Updated!')
    
    #For criminal cases
    crim_sheet.update([current_crim_df.columns.values.tolist()] + current_crim_df.values.tolist())
    print('Criminal Cases Updated!')
    
    return
    
    

In [44]:
#For Testing
#Set up path_list
#Create a list of PDF paths to iterate through
path_list = ['/Users/johnathonsmith/Downloads/CV.PEND.CASE_Dimmit_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CV.PEND.CASE_Zavala_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CV.PEND.CASE_MAVERICK_2-3-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Zavala_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Maverick_1-21-2022.pdf',
             '/Users/johnathonsmith/Downloads/CR.PEND_Dimmit_1-21-2022.pdf']

In [45]:
#Call function
update_spreadsheet(path_list)

Collected Data From 644 Cases.
Collected Data From 779 Cases.
Collected Data From 1110 Cases.
Collected Data From 59 Cases.
Collected Data From 221 Cases.
Collected Data From 77 Cases.
Civil Cases Updated!
Criminal Cases Updated!
